# Te Ataarangi Lesson

This notebook attempts to implement a lesson in Te Ataarangi using the Rational Speech Act framework.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary cache directory at /tmp/matplotlib-3fhjugkm because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def normalize(x):
    return x / np.sum(x, axis=0)

def safe_log(x, eps=1e-10):
    clipped_x = np.clip(x, eps, None)
    return np.where(x > 0, np.log(clipped_x), -np.inf)

def matrices_are_similar(M, N, tolerance=0.1):
    """
    Check if two matrices are similar within a certain tolerance.
    """
    return np.allclose(M, N, atol=tolerance)

In [9]:
class RationalSpeechAgent:
    
    def __init__(self, world_states, utterances, literal_listener_matrix, prior=None):
        self.world_states = world_states
        self.utterances = utterances
        self.prior = prior if prior is not None else np.ones(len(world_states)) / len(world_states)
        self.literal_listener_matrix = normalize(literal_listener_matrix)

    def literal_listener(self, utterance_index):
        return self.literal_listener_matrix[:, utterance_index]

    def pragmatic_speaker(self, world_state_index, alpha=1.0, cost=0):
        U = lambda u: safe_log(alpha * self.literal_listener_matrix[u, :]) - cost
        return normalize(np.exp(alpha * U(world_state_index)))

    def pragmatic_listener(self, utterance_index):
        pragmatic_speaker_matrix = np.array([self.pragmatic_speaker(index) for index, world_state in enumerate(self.world_states)])
        return normalize(pragmatic_speaker_matrix[:, utterance_index] * self.prior)

# Instantiate the agent with the given world states and utterances
agent = RationalSpeechAgent(
    world_states=['1 rākau', '2 rākau', '3 rākau', '4 rākau'],
    utterances=['te rākau', 'ngā rākau'],
    literal_listener_matrix=[
            [1.0, 0.0],  # 1 rākau
            [0.0, 1.0],  # 2 rākau
            [0.0, 1.0],  # 3 rākau
            [0.0, 1.0],  # 4 rākau
    ]
)

# Test the literal listener's output
print("Testing Literal Listener")
for idx, utterance in enumerate(agent.utterances):
    result = agent.literal_listener(idx)
    print(f"Literal listener probabilities for '{utterance}': {result}")

print("\nTesting Pragmatic Speaker")
for idx, state in enumerate(agent.world_states):
    result = agent.pragmatic_speaker(idx)
    print(f"Pragmatic speaker probabilities for world state '{state}': {result}")

print("\nTesting Pragmatic Listener")
for idx, utterance in enumerate(agent.utterances):
    result = agent.pragmatic_listener(idx)
    print(f"Pragmatic listener updated beliefs for utterance '{utterance}': {result}")

Testing Literal Listener
Literal listener probabilities for 'te rākau': [1. 0. 0. 0.]
Literal listener probabilities for 'ngā rākau': [0.         0.33333333 0.33333333 0.33333333]

Testing Pragmatic Speaker
Pragmatic speaker probabilities for world state '1 rākau': [1. 0.]
Pragmatic speaker probabilities for world state '2 rākau': [0. 1.]
Pragmatic speaker probabilities for world state '3 rākau': [0. 1.]
Pragmatic speaker probabilities for world state '4 rākau': [0. 1.]

Testing Pragmatic Listener
Pragmatic listener updated beliefs for utterance 'te rākau': [1. 0. 0. 0.]
Pragmatic listener updated beliefs for utterance 'ngā rākau': [0.         0.33333333 0.33333333 0.33333333]


In [11]:
import numpy as np

def normalize(x):
    return x / np.sum(x, axis=0)

def safe_log(x, eps=1e-10):
    clipped_x = np.clip(x, eps, None)
    return np.where(x > 0, np.log(clipped_x), -np.inf)

class RationalSpeechAgent:
    def __init__(self, world_states, utterances, literal_listener_matrix, prior=None):
        self.world_states = world_states
        self.utterances = utterances
        self.literal_listener_matrix = normalize(np.array(literal_listener_matrix))
        self.prior = prior if prior is not None else np.ones(len(world_states)) / len(world_states)

    def literal_listener(self, utterance_index):
        return self.literal_listener_matrix[:, utterance_index]

    def pragmatic_speaker(self, world_state_index, alpha=1.0):
        utilities = np.array([safe_log(alpha * probability) for probability in self.literal_listener_matrix[world_state_index, :]])
        return normalize(np.exp(utilities))

    def pragmatic_listener(self, utterance_index):
        speaker_beliefs = np.array([self.pragmatic_speaker(ws) for ws in range(len(self.world_states))])
        return normalize(np.dot(speaker_beliefs.T, self.prior)[utterance_index])

# Define a teacher with the correct mappings and a student starting with incorrect or flat priors
teacher = RationalSpeechAgent(
    world_states=['1 rākau', '2 rākau', '3 rākau', '4 rākau'],
    utterances=['te rākau', 'ngā rākau'],
    literal_listener_matrix=[
        [1.0, 0.0],  # 1 rākau
        [0.0, 1.0],  # 2 rākau
        [0.0, 1.0],  # 3 rākau
        [0.0, 1.0],  # 4 rākau
    ]
)

# Student starts with a flat understanding
student = RationalSpeechAgent(
    world_states=['1 rākau', '2 rākau', '3 rākau', '4 rākau'],
    utterances=['te rākau', 'ngā rākau'],
    literal_listener_matrix=[
        [0.5, 0.5],  # 1 rākau
        [0.5, 0.5],  # 2 rākau
        [0.5, 0.5],  # 3 rākau
        [0.5, 0.5],  # 4 rākau
    ]
)

print("Teaching Session:")
for world_state_index, world_state in enumerate(teacher.world_states):
    # Find the correct utterance for the given world state based on the teacher's model
    # This part assumes the teacher has a deterministic way of selecting the utterance,
    # which is the most probable one according to its own literal listener matrix.
    utterance_probabilities = teacher.pragmatic_speaker(world_state_index)
    utterance_index = np.argmax(utterance_probabilities)
    utterance = teacher.utterances[utterance_index]
    
    # Student updates its belief based on the teacher's utterance
    # Here we directly update the student's literal listener matrix 
    # because we are using a supervised, example-based learning approach.
    student.literal_listener_matrix[:, utterance_index] = 0  # Reset other associations
    student.literal_listener_matrix[world_state_index, utterance_index] = 1  # Set the observed association

    print(f"Teacher shows '{world_state}' and says '{utterance}'. Student learns the association.")

# Test the student's learning
print("\nTesting Student's Learning")
for world_state_index, world_state in enumerate(student.world_states):
    student_conjecture_index = np.argmax(student.pragmatic_speaker(world_state_index))
    student_conjecture = student.utterances[student_conjecture_index]
    print(f"For world state '{world_state}', student now says '{student_conjecture}'.")


Teaching Session:
Teacher shows '1 rākau' and says 'te rākau'. Student learns the association.
Teacher shows '2 rākau' and says 'ngā rākau'. Student learns the association.
Teacher shows '3 rākau' and says 'ngā rākau'. Student learns the association.
Teacher shows '4 rākau' and says 'ngā rākau'. Student learns the association.

Testing Student's Learning
For world state '1 rākau', student now says 'te rākau'.
For world state '2 rākau', student now says 'te rākau'.
For world state '3 rākau', student now says 'te rākau'.
For world state '4 rākau', student now says 'ngā rākau'.


/tmp/ipykernel_4032/976681710.py:4: RuntimeWarning: invalid value encountered in divide
  return x / np.sum(x, axis=0)


In [33]:
import numpy as np

class RationalSpeechAgent:
    def __init__(self, world_states, utterances, literal_listener_matrix, prior=None):
        self.world_states = world_states
        self.utterances = utterances
        self.literal_listener_matrix = normalize(np.array(literal_listener_matrix))
        self.prior = prior if prior is not None else np.ones(len(world_states)) / len(world_states)

    def literal_listener(self, utterance_index):
        return self.literal_listener_matrix[:, utterance_index]

    def pragmatic_speaker(self, world_state_index, alpha=1.0):
        utilities = np.array([safe_log(alpha * probability) for probability in self.literal_listener_matrix[world_state_index, :]])
        return normalize(np.exp(utilities))

    def pragmatic_listener(self, utterance_index):
        speaker_matrix = np.array([self.pragmatic_speaker(ws) for ws in range(len(self.world_states))])
        return normalize(np.dot(speaker_matrix.T, self.prior)[utterance_index])

class TeacherAgent(RationalSpeechAgent):
    def __init__(self, world_states, utterances, literal_listener_matrix, student_model_matrix, prior=None):
        super().__init__(world_states, utterances, literal_listener_matrix, prior)
        self.student_model_matrix = normalize(np.array(student_model_matrix))  # Teacher's belief about the student's knowledge

    def update_student_model(self, student_utterance_index, world_state_index):
        # Here, you would update the teacher's model of the student's knowledge based on the student's responses
        # This is a simple heuristic: if the student's response is correct, increase confidence in that mapping
        correct_utterance_index = np.argmax(self.literal_listener_matrix[world_state_index, :])
        if student_utterance_index == correct_utterance_index:
            self.student_model_matrix[world_state_index, student_utterance_index] += 0.1  # Reinforce the correct mapping
        self.student_model_matrix = normalize(self.student_model_matrix)

    def suggest_world_state(self):
        """
        Identify the world state where the teacher believes the student needs the most guidance.
        We look for the biggest difference in the probability distributions for each world state.
        """
        diff_matrix = np.abs(self.literal_listener_matrix - self.student_model_matrix)
        world_state_index = np.argmax(diff_matrix.sum(axis=1))  # Find the index with the largest difference in distributions
        return world_state_index


class StudentAgent(RationalSpeechAgent):
    def __init__(self, world_states, utterances, literal_listener_matrix, prior=None):
        super().__init__(world_states, utterances, literal_listener_matrix, prior)
    
    def update_belief(self, world_state_index, observed_utterance_index):
        # When the student receives an (utterance, world state) pair, it updates its belief.
        # Here, we're simplifying the update rule to directly adopt the teacher's example.
        # This means setting the probability of the observed utterance for the given world state to 1
        # and adjusting other utterance probabilities for that state accordingly.

        # Reset current state's utterance probabilities
        self.literal_listener_matrix[world_state_index, :] = 0.0
        # Set the observed utterance probability to 1 for the given world state
        self.literal_listener_matrix[world_state_index, observed_utterance_index] = 1.0

        # Normalize the matrix if needed to ensure it still represents valid probabilities
        # In this case, since we're setting a single utterance to 1 and others to 0,
        # normalization might not be necessary, but it's good practice to include this step
        # in case the update rule becomes more complex in the future.
        self.literal_listener_matrix = normalize(self.literal_listener_matrix)

In [36]:
# Instantiate teacher and student
teacher = TeacherAgent(
    world_states=['1 rākau', '2 rākau', '3 rākau', '4 rākau'],
    utterances=['te rākau', 'ngā rākau'],
    literal_listener_matrix=[
        [1.0, 0.0],  # 1 rākau
        [0.0, 1.0],  # 2 rākau
        [0.0, 1.0],  # 3 rākau
        [0.0, 1.0],  # 4 rākau
    ],
    student_model_matrix=np.full((4, 2), 0.5)  # Teacher's initial model of the student's knowledge
)

student = StudentAgent(
    world_states=['1 rākau', '2 rākau', '3 rākau', '4 rākau'],
    utterances=['te rākau', 'ngā rākau'],
    literal_listener_matrix=np.full((4, 2), 0.5)  # Student starts with no specific knowledge
)

interaction_count = 0
while not matrices_are_similar(teacher.student_model_matrix, teacher.literal_listener_matrix, tolerance=0.01):
    interaction_count += 1
    print(f"\nInteraction {interaction_count}:")

    # Teacher determines which world state to demonstrate based on where the student needs most guidance
    world_state_index = teacher.suggest_world_state()
    teacher_utterance_index = np.argmax(teacher.pragmatic_speaker(world_state_index))
    teacher_utterance = teacher.utterances[teacher_utterance_index]

    # Teacher demonstrates
    print(f"Teacher: For '{teacher.world_states[world_state_index]}', the best utterance is '{teacher_utterance}'.")

    # Student observes and updates its belief
    student.update_belief(world_state_index, teacher_utterance_index)

    # Student's turn to conjecture
    student_world_state_index = np.random.randint(0, len(student.world_states))  # Random for now, but could also be strategic
    student_utterance_index = np.argmax(student.pragmatic_speaker(student_world_state_index))
    student_utterance = student.utterances[student_utterance_index]

    # Teacher updates its model of the student
    teacher.update_student_model(student_utterance_index, student_world_state_index)

    # Feedback (optional but useful for observation and for student's adjustment in an expanded model)
    # Feedback (optional but useful for observation and for student's adjustment in an expanded model)
    correct_utterance_index = np.argmax(teacher.pragmatic_speaker(student_world_state_index))
    if student_utterance_index == correct_utterance_index:
        print(f"Student: For '{student.world_states[student_world_state_index]}', I believe the correct utterance is '{student_utterance}'. Correct!")
    else:
        correct_utterance = teacher.utterances[correct_utterance_index]
        print(f"Student: For '{student.world_states[student_world_state_index]}', I believe the correct utterance is '{student_utterance}'. Incorrect. The correct utterance should be '{correct_utterance}'.")

    if interaction_count > 1000:  # Safety break to avoid infinite loops in case of convergence issues
        print("Interaction limit reached without convergence.")
        break

if interaction_count <= 1000:
    print("\nThe student's understanding is now aligned with the teacher's knowledge.")



Interaction 1:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '2 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 2:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '2 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 3:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '4 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 4:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '3 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 5:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '2 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 6:
Teacher: For '1 rākau', the best utterance is 'te rākau'.
Student: For '2 rākau', I believe the correct utterance is 'ngā rākau'. Correct!

Interaction 7:
Teacher: For '1 rākau', the best utterance is 'te rāka

In [37]:
teacher.literal_listener_matrix

array([[1.        , 0.        ],
       [0.        , 0.33333333],
       [0.        , 0.33333333],
       [0.        , 0.33333333]])

In [38]:
student.literal_listener_matrix

array([[1.        , 0.        ],
       [0.        , 0.27350545],
       [0.        , 0.71253675],
       [0.        , 0.0139578 ]])

In [39]:
teacher.literal_listener_matrix - teacher.student_model_matrix

array([[ 1.68240059e-03, -2.52683112e-08],
       [-5.60800198e-04,  3.69510655e-03],
       [-5.60800198e-04, -2.60891577e-03],
       [-5.60800198e-04, -1.08616551e-03]])